In [93]:
import MetaTrader5 as mt5

from atj_trading.backtester import get_ohlc_history, Backtester
from datetime import datetime, timedelta, time

import plotly.express as px 

In [154]:
mt5.initialize()

symbol = 'NZDJPY'
start_dt = datetime.now() - timedelta(days=365*2)
end_dt = datetime.now()

ohlc_df = get_ohlc_history(symbol, mt5.TIMEFRAME_M15, start_dt, end_dt)
ohlc_df

,time,open,high,low,close
0,2022-09-05 00:00:00,85.220,85.311,85.205,85.272
1,2022-09-05 00:15:00,85.272,85.320,85.200,85.237
2,2022-09-05 00:30:00,85.237,85.319,85.209,85.302
3,2022-09-05 00:45:00,85.300,85.300,85.195,85.195
4,2022-09-05 01:00:00,85.233,85.424,85.232,85.319
...,...,...,...,...,...
49682,2024-09-02 19:15:00,91.512,91.513,91.461,91.467
49683,2024-09-02 19:30:00,91.467,91.499,91.449,91.492
49684,2024-09-02 19:45:00,91.492,91.498,91.429,91.442
49685,2024-09-02 20:00:00,91.440,91.476,91.437,91.470


In [155]:
ohlc_df['gain'] = ohlc_df['open'].shift(-1) - ohlc_df['open'] 
ohlc_df['perc_gain'] = (ohlc_df['gain'] / ohlc_df['open']) * 100
ohlc_df['time_hms'] = ohlc_df['time'].dt.time
ohlc_df['year'] = ohlc_df['time'].dt.year
ohlc_df['month'] = ohlc_df['time'].dt.month
ohlc_df['day_of_week'] = ohlc_df['time'].dt.dayofweek

ohlc_df = ohlc_df[ohlc_df['time_hms'].between(time(2, 0, 0), time(23, 0, 0))]

ohlc_df

,time,open,high,low,close,gain,perc_gain,time_hms,year,month,day_of_week
8,2022-09-05 02:00:00,85.452,85.529,85.447,85.502,0.050,0.058512,02:00:00,2022,9,0
9,2022-09-05 02:15:00,85.502,85.530,85.476,85.510,0.008,0.009357,02:15:00,2022,9,0
10,2022-09-05 02:30:00,85.510,85.554,85.475,85.553,0.043,0.050287,02:30:00,2022,9,0
11,2022-09-05 02:45:00,85.553,85.586,85.465,85.498,-0.055,-0.064288,02:45:00,2022,9,0
12,2022-09-05 03:00:00,85.498,85.608,85.496,85.584,0.088,0.102926,03:00:00,2022,9,0
...,...,...,...,...,...,...,...,...,...,...,...
49682,2024-09-02 19:15:00,91.512,91.513,91.461,91.467,-0.045,-0.049174,19:15:00,2024,9,0
49683,2024-09-02 19:30:00,91.467,91.499,91.449,91.492,0.025,0.027332,19:30:00,2024,9,0
49684,2024-09-02 19:45:00,91.492,91.498,91.429,91.442,-0.052,-0.056836,19:45:00,2024,9,0
49685,2024-09-02 20:00:00,91.440,91.476,91.437,91.470,0.030,0.032808,20:00:00,2024,9,0


In [156]:
ohlc_df_by_hour = ohlc_df.groupby(['month', 'time_hms'], as_index=False)['perc_gain'].mean()
ohlc_df_by_hour

,month,time_hms,perc_gain
0,1,02:00:00,-0.002459
1,1,02:15:00,0.010594
2,1,02:30:00,0.003068
3,1,02:45:00,-0.004232
4,1,03:00:00,-0.001961
...,...,...,...
1015,12,22:00:00,0.000970
1016,12,22:15:00,0.005576
1017,12,22:30:00,0.004578
1018,12,22:45:00,0.014430


In [157]:
fig = px.bar(ohlc_df_by_hour, x='time_hms', y='perc_gain', color='month')
fig

In [158]:
ohlc_df_by_hour_day = ohlc_df.groupby(['day_of_week', 'time_hms'], as_index=False)['perc_gain'].mean()
ohlc_df_by_hour_day

,day_of_week,time_hms,perc_gain
0,0,02:00:00,0.004006
1,0,02:15:00,0.011795
2,0,02:30:00,-0.019828
3,0,02:45:00,0.008055
4,0,03:00:00,-0.005572
...,...,...,...
420,4,22:00:00,-0.009411
421,4,22:15:00,0.007825
422,4,22:30:00,-0.002104
423,4,22:45:00,0.006991


In [159]:
fig = px.bar(ohlc_df_by_hour_day, x='time_hms', y='perc_gain', color='day_of_week')
fig

In [216]:
# create trade logic
def on_bar(data, trades, orders):
    
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    
    # entry signal
    if data['time_hms'] in [time(15, 30, 0)] and data['day_of_week'] in [3, 4]:
        print(data['time'], 'buy')
        
        sl = data['open'] - 0.5
        tp = data['open'] + 0.5
        orders.open_trade(symbol, 100000, 'buy', sl=sl, tp=tp)
    
    elif data['time_hms'] == time(15, 15, 0) and data['day_of_week'] in [3, 4]:
        print(data['time'], 'sell')
        
        sl = data['open'] + 0.5
        tp = data['open'] - 0.5
        orders.open_trade(symbol, 100000, 'sell', sl=sl, tp=tp)

    # exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]
        orders.close_trade(trade)
            

In [217]:
# backtest parameters
starting_balance = 100000
currency = 'USD'
exchange_rate = 1 / 150
commission = -15 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(ohlc_df)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades

2022-09-08 15:15:00 sell
2022-09-08 15:30:00 buy
2022-09-09 15:15:00 sell
2022-09-09 15:30:00 buy
2022-09-15 15:15:00 sell
2022-09-15 15:30:00 buy
2022-09-16 15:15:00 sell
2022-09-16 15:30:00 buy
2022-09-22 15:15:00 sell
2022-09-22 15:30:00 buy
2022-09-23 15:15:00 sell
2022-09-23 15:30:00 buy
2022-09-29 15:15:00 sell
2022-09-29 15:30:00 buy
2022-09-30 15:15:00 sell
2022-09-30 15:30:00 buy
2022-10-06 15:15:00 sell
2022-10-06 15:30:00 buy
2022-10-07 15:15:00 sell
2022-10-07 15:30:00 buy
2022-10-13 15:15:00 sell
2022-10-13 15:30:00 buy
2022-10-14 15:15:00 sell
2022-10-14 15:30:00 buy
2022-10-20 15:15:00 sell
2022-10-20 15:30:00 buy
2022-10-21 15:15:00 sell
2022-10-21 15:30:00 buy
2022-10-27 15:15:00 sell
2022-10-27 15:30:00 buy
2022-10-28 15:15:00 sell
2022-10-28 15:30:00 buy
2022-11-03 15:15:00 sell
2022-11-03 15:30:00 buy
2022-11-04 15:15:00 sell
2022-11-04 15:30:00 buy
2022-11-10 15:15:00 sell
2022-11-10 15:30:00 buy
2022-11-11 15:15:00 sell
2022-11-11 15:30:00 buy
2022-11-17 15:15:00 

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,NZDJPY,sell,100000,2022-09-08 15:15:00,87.101,2022-09-08 15:30:00,87.101,87.601,86.601,None,0.00,-15.0,-15.0,-15.0,99985.0
1,closed,NZDJPY,buy,100000,2022-09-08 15:30:00,87.101,2022-09-08 15:45:00,87.092,86.601,87.601,None,-6.00,-15.0,-21.0,-36.0,99964.0
2,closed,NZDJPY,sell,100000,2022-09-09 15:15:00,87.05,2022-09-09 15:30:00,87.038,87.55,86.55,None,8.00,-15.0,-7.0,-43.0,99957.0
3,closed,NZDJPY,buy,100000,2022-09-09 15:30:00,87.038,2022-09-09 15:45:00,86.981,86.538,87.538,None,-38.00,-15.0,-53.0,-96.0,99904.0
4,closed,NZDJPY,sell,100000,2022-09-15 15:15:00,85.937,2022-09-15 15:30:00,85.968,86.437,85.437,None,-20.67,-15.0,-35.67,-131.67,99868.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,closed,NZDJPY,buy,100000,2024-08-23 15:30:00,89.91,2024-08-23 15:45:00,89.905,89.41,90.41,None,-3.33,-15.0,-18.33,1945.34,101945.34
412,closed,NZDJPY,sell,100000,2024-08-29 15:15:00,90.717,2024-08-29 15:30:00,90.728,91.217,90.217,None,-7.33,-15.0,-22.33,1923.01,101923.01
413,closed,NZDJPY,buy,100000,2024-08-29 15:30:00,90.728,2024-08-29 15:45:00,90.933,90.228,91.228,None,136.67,-15.0,121.67,2044.68,102044.68
414,closed,NZDJPY,sell,100000,2024-08-30 15:15:00,91.004,2024-08-30 15:30:00,90.958,91.504,90.504,None,30.67,-15.0,15.67,2060.35,102060.35


In [218]:
pnl_chart = bt.plot_pnl()
pnl_chart